is the line-by-line breakdown of how the code maps to the math:

  1. The Log-Likelihood Equation (PDF Page 4)
  The PDF breaks down the log-probability of a single Gaussian component $\log p(\mathbf{x}|k)$ into three parts:
  $$ \log p(\mathbf{x}|k) = \underbrace{-\log(2\pi)}{\text{Constant}} \underbrace{-\log(\sigma{k1}) - \log(\sigma_{k2})}_{\text{Determinant Term}} \underbrace{- \frac{1}{2} \left[ \frac{(x_1 -
  \mu_{k1})^2}{\sigma_{k1}^2} + \frac{(x_2 - \mu_{k2})^2}{\sigma_{k2}^2} \right]}_{\text{Mahalanobis / Exponential Term}} $$

  2. Mapping Code to Math

  A. The "Mahalanobis" Term (The Bracketed Part)
   * Math: $\frac{(x_1 - \mu_{k1})^2}{\sigma_{k1}^2} + \frac{(x_2 - \mu_{k2})^2}{\sigma_{k2}^2}$
   * Code:

   1     # diff represents (x - μ)
   2     # variances represents σ²
   3     mahalanobis = ((diff ** 2) / variances.unsqueeze(0)).sum(dim=2)
       * diff ** 2 is the numerator $(x - \mu)^2$.
       * / variances divides by the denominator $\sigma^2$.
       * .sum(dim=2) sums the two dimensions ($x_1$ and $x_2$), completing the expression inside the brackets.

  B. The Determinant Term (Log Sigmas)
   * Math: $-\log(\sigma_{k1}) - \log(\sigma_{k2})$
   * Code:

   1     # self.log_variances stores log(σ²) which equals 2*log(σ)
   2     log_det = 0.5 * self.log_variances.sum(dim=1)
       * Summing self.log_variances gives $\log(\sigma_{k1}^2) + \log(\sigma_{k2}^2) = 2\log(\sigma_{k1}) + 2\log(\sigma_{k2})$.
       * Multiplying by 0.5 gives $\log(\sigma_{k1}) + \log(\sigma_{k2})$.
       * Subtracting it later (line 79) matches the negative signs in the equation.

  C. The Constant Term
   * Math: $-\log(2\pi)$
   * Code:

   1     -math.log(2 * math.pi)
       * Note: Ensure you `import math` at the top of your file for this to work.

  D. The Final Mixture (LogSumExp)
   * Math: $\log p(\mathbf{x}) = \log \sum_{k=1}^K \exp [\log p(k) + \log p(\mathbf{x}|k)]$
   * Code:

   1     log_joint = log_pi + log_p_x_given_k  # Inner term: log p(k) + log p(x|k)
   2     log_likelihood = torch.logsumexp(log_joint, dim=1) # Outer term: log sum exp
       * This correctly implements the numerically stable "Log-Sum-Exp" trick required by the PDF to avoid underflow when probabilities are very small.

  Conclusion
  The code is a faithful translation of the PDF's derivation for a 2D GMM with a diagonal covariance matrix.


  Let's separate them to make it crystal clear.

  1. The Probability Definition
  Your intuition is correct.
  The probability of a single sample $x$ is indeed a weighted sum.
  $$p(x) = \sum_{k=1}^{K} \underbrace{p(k)}_{\text{weight } \pi_k} \cdot \underbrace{p(x|k)}_{\text{gaussian density}}$$

   * Theorem Name: This is called the Law of Total Probability.
   * Notation: $p(x|k)$ is the probability density of $x$ assuming it came from Gaussian $k$. Whether you write it as $p(x|k)$ or $\mathcal{N}(x | \mu_k, \Sigma_k)$, it means the same thing.

  ---

  2. The Log-Sum-Exp Trick (The "How")

  You described the process of turning multiplication into addition ($log(a \cdot b) = log(a) + log(b)$). That is standard log-properties.

  The Log-Sum-Exp trick is specifically about step 2: The Sum.

  Here is the problem we are solving:
  We want to calculate the log-likelihood: $\log p(x)$.
  $$ \log p(x) = \log \left( \sum_{k=1}^{K} p(k) \cdot p(x|k) \right) $$

  If $p(k) \cdot p(x|k)$ is tiny (like $10^{-100}$), the computer rounds it to 0 (underflow). If we sum a bunch of zeros, we get 0. $\log(0)$ is $-\infty$. The code crashes.

  Here is the actual 3-step breakdown of Log-Sum-Exp:

  Step 1: Work in Log-Space (The Multiplication fix)
  We never calculate $p(k)$ or $p(x|k)$ directly. We calculate their logs.
  Let $y_k$ be the log-probability of component $k$:
  $$ y_k = \log(p(k)) + \log(p(x|k)) $$
  This is just standard math: turning multiplication into addition.

  Step 2: The Summation Problem
  Now we need to sum them to get $p(x)$, but our values are currently logs. To sum them, we have to exponentiate them back:
  $$ \log p(x) = \log \left( \sum_{k=1}^{K} e^{y_k} \right) $$
  Problem: If $y_k = -1000$, then $e^{-1000}$ underflows to 0. We are back to the start.

  Step 3: The "Shift" Trick (The Actual Log-Sum-Exp)
  We pull out the maximum value ($y_{max}$) from the sum. This is the "magic" algebraic step:
  $$ \log \left( \sum e^{y_k} \right) = \log \left( \sum e^{y_k - y_{max} + y_{max}} \right) = \log \left( e^{y_{max}} \sum e^{y_k - y_{max}} \right) $$
  $$ = y_{max} + \log \left( \sum_{k=1}^{K} e^{y_k - y_{max}} \right) $$

  Why does this work?
   * One of the terms will be $e^{y_{max} - y_{max}} = e^0 = 1$.
   * Since we have at least one $1$ in the sum, the sum is never $0$.
   * $\log(\text{something} \ge 1)$ is always safe (never $-\infty$).

  Summary
   1. Log: Turn multiplication into addition ($ \log \pi + \log \mathcal{N} $).
   2. Sum: We need to sum the probabilities (Law of Total Probability).
   3. Exp (Trick): We subtract the max value before exponentiating, sum them, log them, and add the max value back. This guarantees numerical stability.

✦ Based on the PDF and the code skeleton, here are the specifics for the Gaussian dimensions and covariance:

  1. Dimensionality
  Each Gaussian is 2D. 
   * The data $X$ has shape (n_samples, 2).
   * The means self.means have shape (n_components, 2).

  2. Covariance Matrix
  You do not need to implement a full $2 \times 2$ covariance matrix. The PDF explicitly simplifies this for you.

  Page 2 states: 
  > "In this exercise, we will limit ourselves to a 2D diagonal covariance matrix."

  A diagonal covariance matrix means we assume the dimensions (x and y) are independent. Instead of a matrix, you only need to store the variance for each dimension.

  3. Shape and Parameters
  In your code, the covariance is represented by self.log_variances:
   * Shape: (n_components, 2)
   * Why 2? Because for each of your $K$ Gaussians, you have one variance for the $x$ dimension ($\sigma_{k1}^2$) and one variance for the $y$ dimension ($\sigma_{k2}^2$).

  If this were a "full" covariance model, the shape would be (n_components, 2, 2). But because it is diagonal, the off-diagonal elements are zero, so we only store the 2 diagonal elements per
  component.

  Summary of Shapes:
   * `self.means`: (K, 2) -> Each Gaussian has a center $(x, y)$.
   * `self.log_variances`: (K, 2) -> Each Gaussian has a "spread" in $x$ and a "spread" in $y$.
   * `self.weights`: (K,) -> Each Gaussian has one scalar weight (logit).

 1. The Determinant of a Diagonal Matrix
  First, let's look at your Covariance Matrix, $\Sigma$. Since it is diagonal (as specified in the PDF), it looks like this:

  $$ \Sigma = \begin{bmatrix} \sigma_{1}^2 & 0 \\ 0 & \sigma_{2}^2 \end{bmatrix} $$

  The determinant ($|\Sigma|$) of a diagonal matrix is simply the product of its diagonal elements:
  $$ |\Sigma| = \sigma_{1}^2 \cdot \sigma_{2}^2 $$

  2. The Square Root Term
  The formula has $|\Sigma|^{1/2}$ (the square root of the determinant).
  $$ |\Sigma|^{1/2} = \sqrt{\sigma_{1}^2 \cdot \sigma_{2}^2} = \sigma_{1} \cdot \sigma_{2} $$

  So, the normalization term $\frac{1}{|\Sigma|^{1/2}}$ is actually just $\frac{1}{\sigma_1 \sigma_2}$. This makes intuitive sense: the "volume" of the
  Gaussian is scaled by the spread in dimension 1 times the spread in dimension 2.

  3. Moving to Log-Space
  Now, we take the negative log (because we are calculating log-likelihood):

  $$ -\log(|\Sigma|^{1/2}) $$
  $$ = -\log(\sigma_1 \cdot \sigma_2) $$

  Using the log rule $\log(a \cdot b) = \log(a) + \log(b)$:
  $$ = -(\log(\sigma_1) + \log(\sigma_2)) $$
  $$ = -\log(\sigma_1) - \log(\sigma_2) $$

  4. Connection to Your Code
  In your code, you have self.log_variances.
   * self.log_variances[0] holds $\log(\sigma_1^2)$.
   * self.log_variances[1] holds $\log(\sigma_2^2)$.

  The code does:

   1 0.5 * self.log_variances.sum(dim=1)
  Let's expand that math:
  $$ 0.5 \cdot (\log(\sigma_1^2) + \log(\sigma_2^2)) $$
  Using the log rule $\log(x^2) = 2\log(x)$:
  $$ 0.5 \cdot (2\log(\sigma_1) + 2\log(\sigma_2)) $$
  $$ = \log(\sigma_1) + \log(\sigma_2) $$

  So the code calculates exactly the term derived above!

  Answering your specific questions:

  > "How does log(|sigma_k|0.5) become the two separate values?"**

  Because the determinant of a diagonal matrix is a product, and the log of a product is a sum.

  > "Shouldn't the scaling by var be per feature?"

  Yes, and it is!
   * In the exponential part (Mahalanobis distance), you divide by variance per feature: $\frac{(x-\mu)^2}{\sigma^2}$.
   * In the determinant part (the one we just discussed), you are essentially calculating the "volume" of the Gaussian. The volume is determined by
     multiplying the spread of feature 1 by the spread of feature 2 ($\sigma_1 \cdot \sigma_2$). When you take the log, that multiplication becomes addition


✦ This derivation works out neatly because the covariance matrix $\Sigma$ is diagonal.

  Here is the step-by-step linear algebra that transforms the matrix notation into the simple sum of squared errors.

  1. The Setup
  Let's define our vectors and matrices:
   * Difference Vector: $ \mathbf{d} = (\mathbf{x} - \mathbf{\mu}) = \begin{bmatrix} x_1 - \mu_1 \\ x_2 - \mu_2 \end{bmatrix} $
   * Covariance Matrix (Diagonal): $ \Sigma = \begin{bmatrix} \sigma_1^2 & 0 \\ 0 & \sigma_2^2 \end{bmatrix} $

  2. The Inverse Covariance Matrix ($\Sigma^{-1}$)
  The formula uses the inverse of the covariance matrix.
  For a diagonal matrix, the inverse is simply the reciprocal of the diagonal elements (and the zeros stay zeros).

  $$ \Sigma^{-1} = \begin{bmatrix} \frac{1}{\sigma_1^2} & 0 \\ 0 & \frac{1}{\sigma_2^2} \end{bmatrix} $$

  3. The Matrix Multiplication
  Now we compute the inner part: $(\mathbf{x} - \mathbf{\mu})^T \Sigma^{-1} (\mathbf{x} - \mathbf{\mu})$.

  Step A: Compute $\Sigma^{-1} (\mathbf{x} - \mathbf{\mu})$
  Multiply the inverse matrix by the difference vector:
  $$ \begin{bmatrix} \frac{1}{\sigma_1^2} & 0 \\ 0 & \frac{1}{\sigma_2^2} \end{bmatrix} \begin{bmatrix} x_1 - \mu_1 \\ x_2 - \mu_2 \end{bmatrix} =
  \begin{bmatrix} \frac{x_1 - \mu_1}{\sigma_1^2} \\ \frac{x_2 - \mu_2}{\sigma_2^2} \end{bmatrix} $$
  Notice how the zeros in the matrix prevented $x_1$ and $x_2$ from mixing.

  Step B: Compute the Dot Product
  Now multiply the row vector $(\mathbf{x} - \mathbf{\mu})^T$ by the result from Step A:
  $$ \begin{bmatrix} (x_1 - \mu_1) & (x_2 - \mu_2) \end{bmatrix} \cdot \begin{bmatrix} \frac{x_1 - \mu_1}{\sigma_1^2} \\ \frac{x_2 - \mu_2}{\sigma_2^2}
  \end{bmatrix} $$

  $$ = (x_1 - \mu_1) \cdot \frac{(x_1 - \mu_1)}{\sigma_1^2} + (x_2 - \mu_2) \cdot \frac{(x_2 - \mu_2)}{\sigma_2^2} $$

  $$ = \frac{(x_1 - \mu_1)^2}{\sigma_1^2} + \frac{(x_2 - \mu_2)^2}{\sigma_2^2} $$

  4. The Final Result
  Multiply by the $-0.5$ from the outside:
  $$ -\frac{1}{2} \left( \frac{(x_1 - \mu_1)^2}{\sigma_1^2} + \frac{(x_2 - \mu_2)^2}{\sigma_2^2} \right) $$

  Key Takeaway:
  If the covariance matrix were not diagonal (i.e., if the off-diagonal zeros were numbers like $0.5$), you would have ended up with "cross terms" like $(x_1
  - \mu_1)(x_2 - \mu_2)$. Because we assumed a diagonal covariance, those terms disappear, and the formula simplifies to just the sum of the normalized
  squared distances for each dimension independently.


# UMM

✦ Just as we decomposed the GMM, let's break down the Uniform Mixture Model (UMM) using the exact same logic.

  The high-level structure remains identical:
   1. Log: Work in log-space.
   2. Sum: Law of Total Probability (weighted sum of components).
   3. Exp: Use Log-Sum-Exp for stability.

  The only thing that changes is Step 1: How we calculate $\log p(\mathbf{x}|k)$ (the probability of a point given a specific component).

  ---

  1. The Density Derivation: From Area to Log-Prob

  In a GMM, the probability depends on distance from the mean.
  In a UMM, the probability is constant if you are inside the box, and zero if you are outside.

  A. The "Inside" Case
   * Geometry: A 2D uniform distribution is a rectangle (box).
   * Parameters: Center $\mathbf{c} = (c_1, c_2)$, Size $\mathbf{s} = (s_1, s_2)$.
   * Area: $Area = s_1 \times s_2$.
   * Probability Density: To be a valid distribution, the volume under the square must sum to 1. Since the height is constant:
      $$ p(\mathbf{x}|k) = \frac{1}{\text{Area}} = \frac{1}{s_1 \cdot s_2} $$
   * Log-Probability:
      $$ \log p(\mathbf{x}|k) = \log(1) - \log(s_1 \cdot s_2) $$
      $$ = 0 - (\log s_1 + \log s_2) $$
      $$ = -\log s_1 - \log s_2 $$

  Connection to Code:
  Your model has self.log_sizes (which stores $\log s$).
  So, the log-probability is simply:

   1 # Summing the log_sizes for both dimensions
   2 log_prob_inside = -self.log_sizes.sum(dim=1) 
  Note: This is mathematically identical to the GMM determinant term, but without the $2\pi$ or the Mahalanobis distance.

  ---

  2. The "Hard" Switch (The Mask)

  This is the biggest difference. GMMs trail off to infinity; UMMs hit a wall.

  The Condition:
  A point $x$ is inside the box if it is within the range $[c - \frac{s}{2}, c + \frac{s}{2}]$ for both dimensions.

  Mathematically, it's easier to check the distance from the center:
  $$ |x_{dim} - c_{dim}| \le \frac{s_{dim}}{2} $$

  The Log-Likelihood Logic:
  $$ \log p(\mathbf{x}|k) = \begin{cases} -\log s_1 - \log s_2 & \text{if inside} \\ -\infty & \text{if outside} \end{cases} $$

  In computer code, we replace $-\infty$ with a large negative number (like -1e6) to avoid NaNs during optimization.

  ---

  3. The Forward Implementation Plan

  Here is how the concepts map to the tensor operations you will need:

   1. Expand Dimensions: Just like GMM, you need (N, 1, 2) for data and (1, K, 2) for centers/sizes to compute all pairs.
   2. Check Boundaries (The Mask):
       * Calculate distance: dist = |x - center|
       * Calculate limit: limit = size / 2 (remember size = exp(log_size))
       * Check if dist <= limit for both dimensions.
   3. Apply Logic:
       * Start with the "Inside" probability (calculated in step 1).
       * Where the mask says "outside", overwrite the value with -1e6.
   4. Mixture (Same as GMM): Add log_pi, then logsumexp.

  Summary Comparison


  ┌───────────────────┬─────────────────────────────────────────────────┬───────────────────────────────────────────────┐
  │ Concept           │ GMM                                             │ UMM                                           │
  ├───────────────────┼─────────────────────────────────────────────────┼───────────────────────────────────────────────┤
  │ Probability Shape │ Bell Curve (Smooth)                             │ Box (Hard edges)                              │
  │ "Distance" Term   │ Mahalanobis (Quadratic decay)                   │ Mask (Binary check: Inside/Outside)           │
  │ Normalization     │ $-\log(\sigma_1) - \log(\sigma_2) - \text{const}$ │ $-\log(s_1) - \log(s_2)$                        │
  │ Outside Bounds    │ Probability gets tiny, but $>0$                 │ Probability is exactly $0$ ($\log = -\infty$) │
  └───────────────────┴─────────────────────────────────────────────────┴───────────────────────────────────────────────┘